In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests

In [2]:
## just a few testing to see if we can get anything going with bs4 to find countries...
BASE_URL = "http://en.wikipedia.org/wiki/"
DATAPATH = "data/"
data_total = pd.read_csv(DATAPATH+"cleaned_unique_index.csv",low_memory=False)

data_total = data_total.dropna(subset=["product_name"])

In [3]:
data_total.set_index("code",inplace=True)

In [45]:
from difflib import SequenceMatcher

def similars_strings(s1,s2,threshold=0.6):
    return SequenceMatcher(a=s1,b=s2).ratio() >  threshold

In [69]:
def page_exists(article):
    article = article.replace(" ", "_")
    exists = True
    r = requests.get(BASE_URL+article)
    bs = BeautifulSoup(r.text,'html.parser')
    if not(bs.title.string == "Not Found"):
        get_bs = bs.findAll("b")
        for b in get_bs: 
            if b.string and similars_strings(b.string,notFoundWiki):
                exists= False
            
    return exists


In [105]:

origins = ["Country", "Origin", "Production", "City","Headquarters"]
def try_get_localisation(article):
    if page_exists(article):
        r = requests.get(BASE_URL+article)
        bs = BeautifulSoup(r.text,'html.parser')
        for b in bs.findAll("th") :
            if b.string in origins:
                return b.find_next("a").string
            
        
    return False
            
            
try_get_localisation("Nestle")

'Vevey'

In [112]:
%%time
xs = data_total.product_name[:]
notFoundWiki = "Wikipedia does not have an article with this exact name."
results = []
cnt = 0 
perc10 = round(len(xs)/100)
for val in xs.values:
    if cnt % perc10 == 0 : 
        percent = cnt/len(xs) * 100 
        print("Done : %f %% " % percent)
    if try_get_localisation(val):
        local = (try_get_localisation(val))
        results.append((val, local))
    cnt = cnt + 1
        
        
results

Done : 0.000000 % 
Done : 1.000016 % 
Done : 2.000031 % 
Done : 3.000047 % 
Done : 4.000063 % 
Done : 5.000078 % 
Done : 6.000094 % 
Done : 7.000109 % 
Done : 8.000125 % 
Done : 9.000141 % 
Done : 10.000156 % 


ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /wiki/100%_Juice_Blend,_Cranberry_&_Grape (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000029FDE5DACF8>: Failed to establish a new connection: [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu',))

In [114]:
cnt, len(xs)

(64827, 639490)

<p> After running it for a night(8h~) we only did 10%... So this is clearly not a satisfying way to try and solve our issue. moreover as seen below there are alot of values that do not seem correct at all..</p>

In [116]:
results

[("Reese's Peanut Butter Cups", 'United States'),
 ('Persil', 'Germany'),
 ('Bresaola', 'Argentine beef'),
 ('After Dinner', 'Australia'),
 ('Hot Chocolate', 'West Hampstead'),
 ('Half & Half', 'Yvette Lee Bowser'),
 ('Broccoli', '[1]'),
 ('M&S', 'London'),
 ('Gold Bunny', 'Kilchberg'),
 ('Tic Tac', 'Italy'),
 ('coconut milk', 'Production'),
 ('Mixed Nuts', '[1]'),
 ('Mixed Nuts', '[1]'),
 ('Honey', 'Boilery'),
 ('Whoppers', 'Leaf Brands'),
 ('Good & Plenty', 'Philadelphia'),
 ('Mixed Nuts', '[1]'),
 ('Whole Milk', 'Ireland'),
 ('Whole Milk', 'Ireland'),
 ('Milk', 'Ireland'),
 ('Half & Half', 'Yvette Lee Bowser'),
 ('Milk', 'Ireland'),
 ('Milk', 'Ireland'),
 ('Half & Half', 'Yvette Lee Bowser'),
 ('Creamy', 'Denmark'),
 ('Ground Beef', 'Argentine beef'),
 ('Creamy', 'Denmark'),
 ('Worcestershire Sauce', 'Worcester, England'),
 ('Whole Milk', 'Ireland'),
 ('Half & Half', 'Yvette Lee Bowser'),
 ('Honey', 'Boilery'),
 ('Honey', 'Boilery'),
 ('Coconut Oil', 'Production'),
 ('Basmati Rice',